In [2]:
import numpy as np
import pandas as pd
import re
import dictionaries

In [3]:
df = pd.read_csv('TwitterSentimentAnalysisDataset.csv',encoding='windows-1252',names=['Target','ID','Date','Flag','User','Text'])

Exploratory Data Analysis

In [4]:
df.shape

(1600000, 6)

In [5]:
df.head()

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
df.sample(n=10,random_state=15)

,Target,ID,Date,Flag,User,Text
561460,0,2205426244,Wed Jun 17 04:42:48 PDT 2009,NO_QUERY,BurntCaramel,@mjsluijs Nah.. Most of the antivirus program...
1570185,4,2188543728,Mon Jun 15 22:05:33 PDT 2009,NO_QUERY,gab_iii,@zacseif fly to arizona and we'll have video g...
58096,0,1685820960,Sun May 03 02:15:21 PDT 2009,NO_QUERY,NoSheds,@stephenfry I think you killed mobypic - Site ...
283874,0,1993027493,Mon Jun 01 09:42:50 PDT 2009,NO_QUERY,carebearrr,@mileycyrus and we're gonna miss you guys so ...
34009,0,1564669484,Mon Apr 20 04:29:22 PDT 2009,NO_QUERY,timaree,so tired this morning
801000,4,1468005634,Mon Apr 06 23:13:16 PDT 2009,NO_QUERY,MerCleve,@matt_the_great forget the rest...You need to ...
1066153,4,1965220208,Fri May 29 15:32:16 PDT 2009,NO_QUERY,jemcsh,"@tommcfly ur prince harry came to the usa, so ..."
796932,0,2327967353,Thu Jun 25 09:00:32 PDT 2009,NO_QUERY,evestirling,"@DrJoolz thanks, I went there before...it does..."
1345228,4,2043955672,Fri Jun 05 08:39:37 PDT 2009,NO_QUERY,jclanger,change of plans sims party tonight and then t...
415064,0,2060975737,Sat Jun 06 19:56:30 PDT 2009,NO_QUERY,Valv30,@Twisted4Jordan I am sad bout that. But this ...


In [7]:
df.columns

Index(['Target', 'ID', 'Date', 'Flag', 'User', 'Text'], dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Target  1600000 non-null  int64 
 1   ID      1600000 non-null  int64 
 2   Date    1600000 non-null  object
 3   Flag    1600000 non-null  object
 4   User    1600000 non-null  object
 5   Text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [9]:
df['Flag'].nunique()

1

In [10]:
df = df.drop('Flag',axis=1)

In [11]:
df

,Target,ID,Date,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [12]:
df.dtypes

Target     int64
ID         int64
Date      object
User      object
Text      object
dtype: object

In [13]:
df['Date'].head()

0    Mon Apr 06 22:19:45 PDT 2009
1    Mon Apr 06 22:19:49 PDT 2009
2    Mon Apr 06 22:19:53 PDT 2009
3    Mon Apr 06 22:19:57 PDT 2009
4    Mon Apr 06 22:19:57 PDT 2009
Name: Date, dtype: object

In [14]:
df['Date']

0          Mon Apr 06 22:19:45 PDT 2009
1          Mon Apr 06 22:19:49 PDT 2009
2          Mon Apr 06 22:19:53 PDT 2009
3          Mon Apr 06 22:19:57 PDT 2009
4          Mon Apr 06 22:19:57 PDT 2009
                       ...             
1599995    Tue Jun 16 08:40:49 PDT 2009
1599996    Tue Jun 16 08:40:49 PDT 2009
1599997    Tue Jun 16 08:40:49 PDT 2009
1599998    Tue Jun 16 08:40:49 PDT 2009
1599999    Tue Jun 16 08:40:50 PDT 2009
Name: Date, Length: 1600000, dtype: object

In [15]:
df['Date'] = df['Date'].str.replace(' PDT 2009','')

In [16]:
df.head()

,Target,ID,Date,User,Text
0,0,1467810369,Mon Apr 06 22:19:45,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57,Karoli,"@nationwideclass no, it's not behaving at all...."


In [17]:
df[['Day','Month','Date','Time']]=df['Date'].str.split(pat=" ",expand=True)

In [18]:
df

,Target,ID,Date,User,Text,Day,Month,Time
0,0,1467810369,06,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon,Apr,22:19:45
1,0,1467810672,06,scotthamilton,is upset that he can't update his Facebook by ...,Mon,Apr,22:19:49
2,0,1467810917,06,mattycus,@Kenichan I dived many times for the ball. Man...,Mon,Apr,22:19:53
3,0,1467811184,06,ElleCTF,my whole body feels itchy and like its on fire,Mon,Apr,22:19:57
4,0,1467811193,06,Karoli,"@nationwideclass no, it's not behaving at all....",Mon,Apr,22:19:57
...,...,...,...,...,...,...,...,...
1599995,4,2193601966,16,AmandaMarie1028,Just woke up. Having no school is the best fee...,Tue,Jun,08:40:49
1599996,4,2193601969,16,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,Tue,Jun,08:40:49
1599997,4,2193601991,16,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,Tue,Jun,08:40:49
1599998,4,2193602064,16,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,Tue,Jun,08:40:49


In [19]:
df = df.drop('Date',axis=1)

In [20]:
df

,Target,ID,User,Text,Day,Month,Time
0,0,1467810369,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon,Apr,22:19:45
1,0,1467810672,scotthamilton,is upset that he can't update his Facebook by ...,Mon,Apr,22:19:49
2,0,1467810917,mattycus,@Kenichan I dived many times for the ball. Man...,Mon,Apr,22:19:53
3,0,1467811184,ElleCTF,my whole body feels itchy and like its on fire,Mon,Apr,22:19:57
4,0,1467811193,Karoli,"@nationwideclass no, it's not behaving at all....",Mon,Apr,22:19:57
...,...,...,...,...,...,...,...
1599995,4,2193601966,AmandaMarie1028,Just woke up. Having no school is the best fee...,Tue,Jun,08:40:49
1599996,4,2193601969,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,Tue,Jun,08:40:49
1599997,4,2193601991,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,Tue,Jun,08:40:49
1599998,4,2193602064,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,Tue,Jun,08:40:49


In [21]:
df['Target'].unique()

array([0, 4], dtype=int64)

It can be seen that Target=0 represents negative(sad,upset,angry etc.) tweets

In [22]:
df[df['Target']==0].head(10)

,Target,ID,User,Text,Day,Month,Time
0,0,1467810369,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon,Apr,22:19:45
1,0,1467810672,scotthamilton,is upset that he can't update his Facebook by ...,Mon,Apr,22:19:49
2,0,1467810917,mattycus,@Kenichan I dived many times for the ball. Man...,Mon,Apr,22:19:53
3,0,1467811184,ElleCTF,my whole body feels itchy and like its on fire,Mon,Apr,22:19:57
4,0,1467811193,Karoli,"@nationwideclass no, it's not behaving at all....",Mon,Apr,22:19:57
5,0,1467811372,joy_wolf,@Kwesidei not the whole crew,Mon,Apr,22:20:00
6,0,1467811592,mybirch,Need a hug,Mon,Apr,22:20:03
7,0,1467811594,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...,Mon,Apr,22:20:03
8,0,1467811795,2Hood4Hollywood,@Tatiana_K nope they didn't have it,Mon,Apr,22:20:05
9,0,1467812025,mimismo,@twittera que me muera ?,Mon,Apr,22:20:09


While Target=4 represents positive tweets

In [23]:
df[df['Target']!=0].head(10)

,Target,ID,User,Text,Day,Month,Time
800000,4,1467822272,ersle,I LOVE @Health4UandPets u guys r the best!!,Mon,Apr,22:22:45
800001,4,1467822273,becca210,im meeting up with one of my besties tonight! ...,Mon,Apr,22:22:45
800002,4,1467822283,Wingman29,"@DaRealSunisaKim Thanks for the Twitter add, S...",Mon,Apr,22:22:46
800003,4,1467822287,katarinka,Being sick can be really cheap when it hurts t...,Mon,Apr,22:22:46
800004,4,1467822293,_EmilyYoung,@LovesBrooklyn2 he has that effect on everyone,Mon,Apr,22:22:46
800005,4,1467822391,ajarofalmonds,@ProductOfFear You can tell him that I just bu...,Mon,Apr,22:22:47
800006,4,1467822447,vmdavinci,@r_keith_hill Thans for your response. Ihad al...,Mon,Apr,22:22:51
800007,4,1467822465,jessicavaliyi,"@KeepinUpWKris I am so jealous, hope you had a...",Mon,Apr,22:22:48
800008,4,1467822489,emmasaur28,"@tommcfly ah, congrats mr fletcher for finally...",Mon,Apr,22:22:49
800009,4,1467822496,SherylBreuker,@e4VoIP I RESPONDED Stupid cat is helping me ...,Mon,Apr,22:22:49


Keeping only necessary data and discarding the rest

In [24]:
df = df[['Target','User','Text','Day']]

In [25]:
df

,Target,User,Text,Day
0,0,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon
1,0,scotthamilton,is upset that he can't update his Facebook by ...,Mon
2,0,mattycus,@Kenichan I dived many times for the ball. Man...,Mon
3,0,ElleCTF,my whole body feels itchy and like its on fire,Mon
4,0,Karoli,"@nationwideclass no, it's not behaving at all....",Mon
...,...,...,...,...
1599995,4,AmandaMarie1028,Just woke up. Having no school is the best fee...,Tue
1599996,4,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,Tue
1599997,4,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,Tue
1599998,4,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,Tue


In [26]:
df['Target']=df['Target'].replace(4,1)

C:\Users\asus\AppData\Local\Temp\ipykernel_20768\1862341194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Target']=df['Target'].replace(4,1)


In [27]:
df[df['Target']!=0].head(10)

,Target,User,Text,Day
800000,1,ersle,I LOVE @Health4UandPets u guys r the best!!,Mon
800001,1,becca210,im meeting up with one of my besties tonight! ...,Mon
800002,1,Wingman29,"@DaRealSunisaKim Thanks for the Twitter add, S...",Mon
800003,1,katarinka,Being sick can be really cheap when it hurts t...,Mon
800004,1,_EmilyYoung,@LovesBrooklyn2 he has that effect on everyone,Mon
800005,1,ajarofalmonds,@ProductOfFear You can tell him that I just bu...,Mon
800006,1,vmdavinci,@r_keith_hill Thans for your response. Ihad al...,Mon
800007,1,jessicavaliyi,"@KeepinUpWKris I am so jealous, hope you had a...",Mon
800008,1,emmasaur28,"@tommcfly ah, congrats mr fletcher for finally...",Mon
800009,1,SherylBreuker,@e4VoIP I RESPONDED Stupid cat is helping me ...,Mon


In [28]:
df

,Target,User,Text,Day
0,0,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon
1,0,scotthamilton,is upset that he can't update his Facebook by ...,Mon
2,0,mattycus,@Kenichan I dived many times for the ball. Man...,Mon
3,0,ElleCTF,my whole body feels itchy and like its on fire,Mon
4,0,Karoli,"@nationwideclass no, it's not behaving at all....",Mon
...,...,...,...,...
1599995,1,AmandaMarie1028,Just woke up. Having no school is the best fee...,Tue
1599996,1,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,Tue
1599997,1,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,Tue
1599998,1,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,Tue


In [29]:
text, sentiment = list(df['Text']), list(df['Target'])

In [30]:
import time
t = time.time()
preprocessedtext = dictionaries.preprocess(text) #the preprocess function at work
print(f'Text Processing Done.')
print(f'Time taken for text processing: {round(time.time()-t)} seconds')

Text Processing Done.
Time taken for text processing: 342 seconds


In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(preprocessedtext, sentiment, test_size = 0.05, random_state = 0)

In [32]:
X_train, X_test = dictionaries.vectorizes(X_train,X_test)

Vectoriser fitted.


C:\Users\asus\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


No. of feature_words:  500000


Naive Bayes

SVM

Logistic Regression

Naive Bayes

In [33]:
df.head()

,Target,User,Text,Day
0,0,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon
1,0,scotthamilton,is upset that he can't update his Facebook by ...,Mon
2,0,mattycus,@Kenichan I dived many times for the ball. Man...,Mon
3,0,ElleCTF,my whole body feels itchy and like its on fire,Mon
4,0,Karoli,"@nationwideclass no, it's not behaving at all....",Mon


In [34]:
def class_separation(df):
    df=df.sort_values(by='Target').reset_index()
    df_zeros = df.iloc[0:800000]
    df_zeros = df_zeros.drop(['index','level_0'],axis=1)
    df_ones = df.iloc[800000:]
    df_ones = df_ones.drop(['index','level_0'],axis=1)
    seps = {0:df_zeros, 1:df_ones}
    return seps
    

In [35]:
class_dict = class_separation(df)

KeyError: "['level_0'] not found in axis"

In [ ]:
class_dict[0]

In [ ]:
class_dict[1]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(20,10))
sns.histplot(data=df,x='Target',bins=2,hue='Target',palette='bright')
plt.show()

Sumaarizing the Datset

In [ ]:
df.dtypes

In [ ]:
def summarize(df):
    summaries = []
    for col in df.columns:
        if(col.dtype!='o'):
            summaries.append([np.mean(col),np.std(col),len(col)])
    return summaries